In [2]:
import pandas as pd
import os

In [10]:
!ls -l ../Downloads/2012

total 11352848
-rw-r--r--@ 1 Christine  staff   571920592 Jun 18  2019 all_assistance_prime_transactions_1.csv
-rw-r--r--@ 1 Christine  staff   539177067 Jun 18  2019 all_assistance_prime_transactions_2.csv
-rw-r--r--@ 1 Christine  staff   126709796 Jun 18  2019 all_assistance_prime_transactions_3.csv
-rw-r--r--@ 1 Christine  staff  1586725464 Jun 18  2019 all_contracts_prime_transactions_1.csv
-rw-r--r--@ 1 Christine  staff  1594421633 Jun 18  2019 all_contracts_prime_transactions_2.csv
-rw-r--r--@ 1 Christine  staff  1390093893 Jun 18  2019 all_contracts_prime_transactions_3.csv


In [51]:
year=2011

In [52]:
dirs = os.listdir('../Downloads/{}'.format(year))
d = dict()
for i,files in zip(map(str, range(1, len(dirs)+1)), dirs):
    if files.startswith('all_contracts'):
        d['df_'+i] = pd.read_csv("../Downloads/{}/".format(year) + files)
df = pd.concat([d[i] for i in d.keys()], ignore_index=True).reset_index()

/Users/Christine/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,37,47,49,50,57,66,75,76,89,109,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Christine/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (37,39,47,49,50,57,66,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Christine/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,37,39,43,47,49,50,57,66,75,76,89,109,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
df.shape

(3218111, 262)

In [54]:
df['recipient_city_state'] = df['recipient_city_name'] + ', ' + df['recipient_state_code']
df['performance_city_state'] = df['primary_place_of_performance_city_name'] + ', '  \
                               + df['primary_place_of_performance_state_code']
df_city = df[['awarding_agency_name', 'federal_action_obligation', 'recipient_city_name', 
              'recipient_state_code', 'recipient_city_state', 'primary_place_of_performance_city_name', 
              'primary_place_of_performance_state_code', 'performance_city_state']]

In [55]:
df_recipient = df_city.groupby('recipient_city_state') \
.agg({'federal_action_obligation':'sum', 'awarding_agency_name':'count'}) \
.rename(columns={'awarding_agency_name':'recipient_count', 'federal_action_obligation':'recipient_amount'}) \
.reset_index()

df_performance = df_city.groupby('performance_city_state') \
.agg({'federal_action_obligation':'sum', 'awarding_agency_name':'count'}) \
.rename(columns={'awarding_agency_name':'performance_count', 'federal_action_obligation':'performance_amount'}) \
.reset_index()

df_recipient = df_recipient.merge(df_city[['recipient_city_name','recipient_state_code',
                                           'recipient_city_state']],
                                  on='recipient_city_state').drop_duplicates().reset_index(drop=True)

df_performance = df_performance.merge(df_city[['performance_city_state', 
                                               'primary_place_of_performance_city_name', 
                                               'primary_place_of_performance_state_code']],
                                      on='performance_city_state').drop_duplicates().reset_index(drop=True)

In [56]:
df_performance.head()

,performance_city_state,performance_count,performance_amount,primary_place_of_performance_city_name,primary_place_of_performance_state_code
0,"AARONSBURG, PA",1,-77.14,AARONSBURG,PA
1,"ABBEVILLE, GA",3,133500.00,ABBEVILLE,GA
2,"ABBEVILLE, LA",34,49072.10,ABBEVILLE,LA
3,"ABBEVILLE, MS",34,1417914.40,ABBEVILLE,MS
4,"ABBEVILLE, SC",1,202914.00,ABBEVILLE,SC


In [57]:
df_performance.shape

(14179, 5)

In [58]:
df_recipient.head()

,recipient_city_state,recipient_count,recipient_amount,recipient_city_name,recipient_state_code
0,"ABBEVILLE, AL",21,4771049.03,ABBEVILLE,AL
1,"ABBEVILLE, GA",5,133500.00,ABBEVILLE,GA
2,"ABBEVILLE, LA",29,2426886.57,ABBEVILLE,LA
3,"ABBEVILLE, MS",1,19800.00,ABBEVILLE,MS
4,"ABBEVILLE, SC",5,29827.00,ABBEVILLE,SC


In [59]:
df_recipient.shape

(14931, 5)

In [60]:
df_performance.to_csv('Data/FF/FF{}_performance.csv'.format(year))
df_recipient.to_csv('Data/FF/FF{}_recipient.csv'.format(year))